# 3. 連立一次方程式
### 概要

- 連立一次方程式の解法は二つに分類できる
- 消去法（LU分解）
- 反復法

### key-words

- 吐き出し法
- ガウスの消去法
- LU分解
- 逆行列
- ガウス・ザイデル法
- SOR法


In [2]:
!python -V

Python 3.7.4


Import

In [4]:
import numpy as np
import matplotlib.pyplot as plt

### 注意

- numpyを用いた行列表現はarrayを使用する
- `*`演算をするとアダマール積になるが`@`を用いることで通常の行列の積の実行結果を得ることができる
- Scipy.orgにおける`numpy.matrix` documentにも`Note　It is no longer recommended to use this class, even for linear algebra. Instead use regular arrays. The class may be removed in the future.`とのこと

### 吐き出し法

次のルールを使って、未知数を一つずつ消していく手法

$$
\begin{aligned}
2x + 2y + 6z &= 24 \\
3x + 5y + 13z &= 52\\
5x + 8y + 24z &= 93
\end{aligned}
$$
を考えるとき、次の二元配列（行列）に変換する

$$
\left(
\begin{array}{cccc}
2 & 2 & 6 & 24 \\
3 & 5 & 13 & 52\\
5 & 8& 24 & 93
\end{array}
\right)
$$

- ある行をその行に対応するピボット（対角成分）で割る
- ある行から他の行の定数倍を引く
- ある行と他の行を入れ替える

#### Pseudocode

```
1. Start

2. Input the Augmented Coefficients Matrix (A):
	
	For i = 1 to n
		
		For j = 1 to n+1
			
			Read Ai,j
		
		Next j
	
	Next i

3. Apply Gauss Jordan Elimination on Matrix A:
	
	For i = 1 to n
		
		If Ai,i = 0
			
			Print "Mathematical Error!"
			Stop
		
		End If
		
		For j = 1 to n
			
			If i ≠ j 
				
				Ratio = Aj,i/Ai,i
				
				For k = 1 to n+1
				
					Aj,k = Aj,k - Ratio * Ai,k
			
				Next k
				
			End If
			
		Next j
	Next i

4. Obtaining Solution:
	
	For i = 1 to n 
		Xi = Ai,n+1/Ai,i
	Next i

5. Display Solution:
	
	For i = 1 to n
		
		Print Xi
	
	Next i

6. Stop


---------------
Note: All array indexes are assumed to start from 1.
```


### [例題1]

$$
\begin{aligned}
2x + 2y + 6z &= 24 \\
3x + 5y + 13z &= 52\\
5x + 8y + 24z &= 93
\end{aligned}
$$

を解け


In [86]:
def gauss_jordan(A, output_array = None):
    n, m = A.shape
    for i in range(n):
        if A[i, i] == 0:
            raise ValueError('{}-th pivot is zero'.format(i))
        
        A[i, :] = A[i, :]/A[i,i]
        
        for j in range(n):
            if i != j:
                A[j, :] = A[j, :] - A[j,i] * A[i, :]
    if output_array:
        return A
    return A[:, -1]
                

In [92]:
X = np.array(([2, 2, 6, 24] 
             ,[3, 5, 13, 52]
             ,[5, 8, 24, 93]))
gauss_jordan(X, output_array = True)

array([[1, 0, 0, 1],
       [0, 1, 0, 2],
       [0, 0, 1, 3]])

In [97]:
%%timeit
gauss_jordan(X)

46.3 µs ± 12.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [94]:
A, b = X[:, :-1], X[:, -1]

In [95]:
%%timeit
np.linalg.solve(A, b)

17.2 µs ± 4.41 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


- 自作コードは`for loop`をネストで使用しているため、計算時間がとんでもなく遅いことに留意

### ガウスの消去法: Gauss Elimination Method

- 計算時間が短いので広く利用されている
- ガウスの消去法は前進消去（文字を1つずつ消していく操作）と後退代入（1成分ずつ答えを求めていく操作）からなる

#### 前進消去法のゴール

$$
\left(
\begin{array}{cccc}
u_{11} & u_{12} & u_{13} & b_1 \\
0 & u_{22} & u_{23} & b_2\\
0 & 0 & u_{33} & b_3
\end{array}
\right)
$$

- 後退代入は後ろの式から順番にたどっていき，一つずつ変数の値を求めていくだけ

#### ガウス消去法のアルゴリズム

```
[INPUT]
    A = (a[i,j]): 係数行列(n次の正方行列)
    b = (b[i]): 定数ベクトル
    
[OUTPUT]
    x: 解ベクトル
    
[PROCESS]
    (前進消去法)
        各列k = 1, ..., n-1について次の処理を反復する
        (k列の吐き出し)
            各行 i = k+1, ..., nについて次の処理を反復する
            (a[i,k]の掃き出し)
                pivot = a[k,k]
                ratio = a[i,k]/pivot
                for (j == k+1, j<= n, j++):
                    a[i,j] = a[i,j] - a[k, j] * ratio
                b[i] = b[i] - b[k]*ratio
     
     (後退代入)
         for (k = n, k >= 1, k--):
             x[k] = b[k]
             for (j == k+1, j<= n, j++):
                 x[k] = x[k] - a[k,j]*x[j]
             
             x[k] = x[k]/a[k,k]
```



#### Pseudocode for Gauss Elimination Method

<img src = "https://github.com/RyoNakagami/omorikaizuka/blob/master/pseudocode/gauss_elimination.jpg?raw=true">

- Psuedocodeの`maxEl`はpivot == 0に起因する計算エラーを防ぐために実施している
- 絶対値が大きいpivotを探し出し、スイッチしている（ピボット選択 or 部分ピボット選択という）
- pivotの絶対値はスケールに依存するので、通常は各行の絶対値最大の要素でその行の要素を割るというスケーリング前処理をする

### [例題2]
ガウスの消去法を用いて、次の連立一次方程式を解け

$$
\begin{aligned}
2x + 2y + 6z &= 24 \\
3x + 5y + 13z &= 52\\
5x + 8y + 24z &= 93
\end{aligned}
$$

In [245]:
def gauss_elimination(X):
    A = X.copy()
    n, m = A.shape
    for i in range(0, n):
        # Search for maximum 
        maxEl = abs(A[i][i])
        maxRow = i
        for k in range(i+1, n):
            if abs(A[k][i]) > maxEl:
                maxEl = abs(A[k][i])
                maxRow = k

        if i != maxRow:
            tmp = A[i, :].copy()
            A[i, :] = A[maxRow, :]
            A[maxRow, :] = tmp


        # 対角成分以下をゼロ
        for k in range(i+1, n):
            c = -A[k][i]/A[i][i]
            for j in range(i, n+1):
                if i == j:
                    A[k][j] = 0
                else:
                    A[k][j] += c * A[i][j]

    # Solve equation Ax=b
    x = [0 for i in range(n)]
    for i in range(n-1, -1, -1):
        x[i] = A[i][n]/A[i][i]
        for k in range(i-1, -1, -1):
            A[k][n] -= A[k][i] * x[i]
    return x

In [246]:
gauss_elimination(X)

[1.0, 2.0, 3.0]

In [247]:
%%timeit
gauss_elimination(X)

70.1 µs ± 2.13 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### LU分解：LU Decomposition

LU stands for ‘Lower Upper’, and so an LU decomposition of a matrix A is a decomposition so that

$$
A = LU
$$

- 基本的にはGaussian elimination

#### 処理方針

1. 行列Aのほかに、単位行列Iを準備しこの行列をMとする
2. AをGaussian eliminationすると同時に、それぞれの乗数 $m_{ik}$ を行列Mの(i, k)成分として記憶する
3. アルゴリズムが終了したときのMをL, AをUとする


In [248]:
import scipy.linalg as la

A = np.array([[1,3,4],[2,1,3],[4,1,2]])

print(A)

P, L, U = la.lu(A)
print(L@U)
print(P@L@U)

[[1 3 4]
 [2 1 3]
 [4 1 2]]
[[4. 1. 2.]
 [1. 3. 4.]
 [2. 1. 3.]]
[[1. 3. 4.]
 [2. 1. 3.]
 [4. 1. 2.]]


In [249]:
print(P)

[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]


In [250]:
print(L)

[[1.         0.         0.        ]
 [0.25       1.         0.        ]
 [0.5        0.18181818 1.        ]]


In [251]:
print(U)

[[4.         1.         2.        ]
 [0.         2.75       3.5       ]
 [0.         0.         1.36363636]]


### LU DecompositionのPsuedocode

```
function lu_decompose(A: matrix)
    n = nrow(A)
    P = indentity matrix, which is N times N
    //row swap
    for (i = 1; i <= n, i++) do
         maxEl = |A[i,i]|
         maxRow = i
      
         for (k = i + 1; k <= n; k++) dp
             if |A[k, i]| > maxEl then
                 maxEl = A[k, i]
                 maxRow = k
             end if
         end for
         
         A[maxRow, :], A[i, :] = A[i, :], A[maxRow, :]
         P[maxRow , i], P[i, i]  = 1, 0
    end for
    
    L = I, which is N times N identity matrix
    
    for (i = 1; i <= n, i++) do
        for (k = i + 1; k <= n; k++) do
            m = A[k, i]/A[i, i]
            A[k, :] = A[k, :] - A[i, :] * m
            L[k, i] = m
        end for
    end for
    
    return P, L, A
```

In [252]:
def lu_decomposition(X):
    A = X.copy()
    n = A.shape[0]
    P = np.identity(n)
    L = np.identity(n)
    
    for i in range(n):
        maxEl = abs(A[i, i])
        maxRow = i
        
        for k in range(i+1, n):
            if abs(A[k, i]) > maxEl:
                maxEl = abs(A[k, i])
                maxRow = k
                
        if i != maxRow:
            P[i, i], P[i, maxRow], P[maxRow, i], P[maxRow, maxRow] = 0, 1, 1, 0
            tmp = A[i, :].copy()
            A[i, :] = A[maxRow, :]
            A[maxRow, :] = tmp
    for i in range(n):
        for k in range(i + 1, n):
            m = A[k, i]/A[i, i]
            A[k, :] = A[k, :] - A[i, :] * m
            L[k, i] = m
            print(A)
        
    
    return P, L, A
        

### [例題3]

$$
A = \left(\begin{array}{ccc}
2&2&6\\
3&5&13\\
5&8&24
\end{array}\right)
$$
をLU分解せよ

In [253]:
A = np.array(([2., 2., 6.]
             ,[3., 5., 13.]
             ,[5., 8., 24.]))

P, L, U = lu_decomposition(A)
print(L)
print(U)
print(P)

[[ 5.   8.  24. ]
 [ 0.   0.2 -1.4]
 [ 2.   2.   6. ]]
[[ 5.   8.  24. ]
 [ 0.   0.2 -1.4]
 [ 0.  -1.2 -3.6]]
[[  5.    8.   24. ]
 [  0.    0.2  -1.4]
 [  0.    0.  -12. ]]
[[ 1.   0.   0. ]
 [ 0.6  1.   0. ]
 [ 0.4 -6.   1. ]]
[[  5.    8.   24. ]
 [  0.    0.2  -1.4]
 [  0.    0.  -12. ]]
[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [254]:
print(P @ L @ U)

[[ 2.  2.  6.]
 [ 3.  5. 13.]
 [ 5.  8. 24.]]


### LU分解アルゴリズムの正当性
#### 定理

入力Aに対するLU分解のアルゴリズムの出力がL, Uならば$LU = A$

#### [証明]

任意の計算状態$<M_t, A_t>$において$M_tA_t = A$であることを帰納法を用いて示す。

$t = 0$のときは $M_0A_0 = IA = A$より自明。

次に、帰納法の仮定
$$
M_t A_t = A
$$

のもとで

$$
M_{t+1} A_{t+1} = A
$$

を示す。

第$t+1$ stepで掃き出すのが$A_t$の(i, k)成分であるので、アルゴリズムより$ i > k$, このときの乗数を$m_{ik}$とする。ここで単位行列の(i, k)成分をそれぞれ$m_{i,k}, -m_{i,k}$としたものをそれぞれS, Tとする。

$$
S = \left(\begin{array}{cccccc}
1 & 0 & ...& ...& ...& 0\\
0 & ... & ...& ...& ...& 0\\
0 & ... & 1 & ... & ...& ...\\
...& ... & ...& ...& ...& ...\\
0 & ... & m_{ik}& ... & ...& ...\\
0 & ... & ...& 0& ...& 1
\end{array}\right)
$$

$$
T = \left(\begin{array}{cccccc}
1 & 0 & ...& ...& ...& 0\\
0 & ... & ...& ...& ...& 0\\
0 & ... & 1 & ... & ...& ...\\
...& ... & ...& ...& ...& ...\\
0 & ... & -m_{ik}& ... & ...& ...\\
0 & ... & ...& 0& ...& 1
\end{array}\right)
$$

このとき以下の性質を用いる

1. $ST = I$
2. Sを任意の行列Mの右から掛けると、Mの第i列の$m_{ik}$倍をMの第k列に加える操作に相当する

よって
$$
\begin{aligned}
M_{t + 1} &= M_t S\\
A_{t + 1} & = T A_t
\end{aligned}
$$

Then,

$$
M_{t + 1}A_{t + 1} = (M_t S)(T A_t) = M_t A_t = A
$$

よって任意のtについて$M_t A_t = A$. Then $LU = M_{T}A_{T} = A$ where T は最終ステップ。

### LUx = bを解くアルゴリズム

```
[INPUT]
    L, U: LU分解
    B: 定数ベクトル
    
[OUTPUT]
    x: solution
    
[PROCESS]
    n = nrows(L)
    (前進代入)
        y = b
        for (i = 1; i <= n; i++) do
            for (j = 1; j <= n; j++) do
                y[i] = y[i] - L[i, j] * y[j]
            end for
        end for
    
    {後退代入}
        x = y
        for (i = n; i >= 1; i--) do
            for (j = i + 1; i <= n; i++) do
                x[i] = x[i] - U[i, j] * x[j]
            end for
            x[i] = x[i]/U[i, i]
        end for
        
```


array([[ 1. ,  0. ,  0. ],
       [ 0.6,  1. ,  0. ],
       [ 0.4, -6. ,  1. ]])